In [1]:
import time
import json
import time
from riotwatcher import LolWatcher, ApiError, RiotWatcher

In [6]:
# グローバル変数を初期化
request_count = 0  # 現在のリクエスト数
start_time = time.time()  # 開始時間（秒）

def rate_limit_request(request_func, *args, **kwargs):

    global request_count, start_time

    elapsed_time = time.time() - start_time  # 経過時間を計算

    # 1秒あたり20リクエスト制限
    if request_count >= 20 and elapsed_time < 1:
        time.sleep(1 - elapsed_time)  # 必要に応じて1秒まで待機
        start_time = time.time()  # 開始時間をリセット
        request_count = 0  # カウントをリセット

    # 2分あたり100リクエスト制限
    if request_count % 100 == 0 and request_count != 0 and elapsed_time < 120:
        time.sleep(120 - elapsed_time)  # 必要に応じて2分まで待機
        start_time = time.time()  # 開始時間をリセット

    request_count += 1  # リクエストカウントを増加
    return request_func(*args, **kwargs)  # リクエストを実行


In [7]:
api_key = "RGAPI-94bbe2f8-df0d-47c7-8bec-f01712e8ba80"
region = "asia"

Rw = RiotWatcher(api_key)
Lw = LolWatcher(api_key)

In [8]:
# プレイヤー情報（名前とタグライン）
players = [
    {"name": "민철이여친구함", "tagline": "0415"},
    {"name": "M G", "tagline": "2821"},
    {"name": "DK ShowMaker", "tagline": "KR1"},
    {"name": "Yondaime", "tagline": "Luo"},
    {"name": "FATE", "tagline": "KR2"},
]

In [9]:
# 1. PUUIDを取得
puuids = {}  # プレイヤー名をキーにPUUIDを保存
for player in players:
    pu = Rw.account.by_riot_id(region=region, game_name=player['name'], tag_line=player['tagline'])
    puuids[player['name']] = pu['puuid'] 
    print(f" {player['name']}: {pu['puuid']}")


 민철이여친구함: Tg6htH2UT_o9o11TbmE11RYbJ-o9Uk4iAWU1iaYOu13gc3V0nriz-vqdHfSrRJmhkBC-2ejHAjZ2fA
 M G: yBObD3pa4qWEzI5v_Uo-azpDPxKgX9c7KbgzeIBw6uQCQmLc3xsZVj8KsEI3JkESDPiYxvKv8MIuDA
 DK ShowMaker: RaFeXYSCommWUge7Uj_eVGna__co3FOav4nXhVOqjPq2dHGYxa-OlFwT7Om0L-w1c9ShIn9JaOt02Q
 Yondaime: _DJNQWYxK5yggUNX6lF40G9fibqJZuSOr-JFrE1sWyGON9afLLJKjtqL6TO4Oz0KloQ9rfLi3MB6_g
 FATE: dXbgFId6t4WGo8XttZ5bcuBq6r-Xycjas9WJNgOMcTR025YA7WM4PO1A3kT-fjYtkyw3v0gFiw8fcg


In [10]:
# 2. 直近500試合のマッチIDを取得
match_ids = {player['name']: [] for player in players}  # プレイヤー名をキーに試合IDリストを保存

for player in players:
    puuid = puuids.get(player['name'])
    for start in range(0, 500, 100):
        # PUUIDを使って試合リストを取得
        matches = Lw.match.matchlist_by_puuid(region=region, puuid=puuid, start=start, count=100)
        match_ids[player['name']].extend(matches)

# 結果を表示
total_matches = sum(len(matches) for matches in match_ids.values())  # 合計試合数を計算

print(f"合計取得された試合ID数: {total_matches}")
for name, matches in match_ids.items():
    print(f"Player: {name}, Match IDの数: {len(matches)}")
    print(f"Match IDs: {matches}")

合計取得された試合ID数: 2500
Player: 민철이여친구함, Match IDの数: 500
Match IDs: ['KR_7425669441', 'KR_7425600873', 'KR_7425572072', 'KR_7425520090', 'KR_7425424768', 'KR_7424700184', 'KR_7423978902', 'KR_7423908127', 'KR_7423839668', 'KR_7423790967', 'KR_7423719857', 'KR_7422447692', 'KR_7415838761', 'KR_7414312604', 'KR_7414278192', 'KR_7414207052', 'KR_7412574012', 'KR_7412549141', 'KR_7412492861', 'KR_7412437288', 'KR_7412390763', 'KR_7412291677', 'KR_7410922107', 'KR_7410801708', 'KR_7410709569', 'KR_7410658377', 'KR_7410561383', 'KR_7410474867', 'KR_7410004148', 'KR_7409953234', 'KR_7409387096', 'KR_7409351851', 'KR_7409336153', 'KR_7409300087', 'KR_7409246232', 'KR_7405167207', 'KR_7403626177', 'KR_7403525827', 'KR_7401937018', 'KR_7401845852', 'KR_7401802761', 'KR_7401623001', 'KR_7400918213', 'KR_7400390069', 'KR_7400361085', 'KR_7400283791', 'KR_7400203858', 'KR_7400033411', 'KR_7399434496', 'KR_7398870981', 'KR_7398801402', 'KR_7398753648', 'KR_7398673196', 'KR_7397990849', 'KR_7397945464', '

In [20]:
# 出力先のJSONファイル
output_file = 'match_data.json'

# JSONファイルを初期化（空のリストを作成）
with open(output_file, 'w', encoding='utf-8') as file:
    file.write('[')  # JSON配列の開始

# 全ての試合IDをリストとして取得
all_match_ids = [match_id for match_list in match_ids.values() for match_id in match_list]

# 全試合データを取得して保存
for index, match_id in enumerate(all_match_ids):
    # 試合データを取得
    match_data = Lw(region=region, match_id=match_id)

    # 試合データを保存
    with open(output_file, 'a', encoding='utf-8') as file:
        if index > 0:  # 最初の試合データ以外はカンマを追加
            file.write(',\n')
        json.dump(match_data, file, ensure_ascii=False, indent=4)

    print(f"Match ID {match_id} 保存完了 ({index + 1}/{len(all_match_ids)})")

# JSON配列の終了部分を追加
with open(output_file, 'a', encoding='utf-8') as file:
    file.write('\n]')  # JSON配列の終了

print("全ての試合データの保存完了")


AttributeError: 'MatchApiV5' object has no attribute 'timeline'

In [11]:
# 出力先のJSONファイル
output_file = 'timeline_data.json'

# JSONファイルを初期化（空のリストを作成）
with open(output_file, 'w', encoding='utf-8') as file:
    file.write('[')  # JSON配列の開始

# 全ての試合IDをリストとして取得
all_match_ids = [match_id for match_list in match_ids.values() for match_id in match_list]

# 全試合データを取得して保存
for index, match_id in enumerate(all_match_ids):
    # 試合データを取得
    timeline_data = Lw.match.timeline_by_match(region=region, match_id=match_id)

    # 試合データを保存
    with open(output_file, 'a', encoding='utf-8') as file:
        if index > 0:  # 最初の試合データ以外はカンマを追加
            file.write(',\n')
        json.dump(timeline_data, file, ensure_ascii=False, indent=4)

    print(f"Match ID {match_id} 保存完了 ({index + 1}/{len(all_match_ids)})")

# JSON配列の終了部分を追加
with open(output_file, 'a', encoding='utf-8') as file:
    file.write('\n]')  # JSON配列の終了

print("全ての試合データの保存完了")

Match ID KR_7425669441 保存完了 (1/2500)
Match ID KR_7425600873 保存完了 (2/2500)
Match ID KR_7425572072 保存完了 (3/2500)
Match ID KR_7425520090 保存完了 (4/2500)
Match ID KR_7425424768 保存完了 (5/2500)
Match ID KR_7424700184 保存完了 (6/2500)
Match ID KR_7423978902 保存完了 (7/2500)
Match ID KR_7423908127 保存完了 (8/2500)
Match ID KR_7423839668 保存完了 (9/2500)
Match ID KR_7423790967 保存完了 (10/2500)
Match ID KR_7423719857 保存完了 (11/2500)
Match ID KR_7422447692 保存完了 (12/2500)
Match ID KR_7415838761 保存完了 (13/2500)
Match ID KR_7414312604 保存完了 (14/2500)
Match ID KR_7414278192 保存完了 (15/2500)
Match ID KR_7414207052 保存完了 (16/2500)
Match ID KR_7412574012 保存完了 (17/2500)
Match ID KR_7412549141 保存完了 (18/2500)
Match ID KR_7412492861 保存完了 (19/2500)
Match ID KR_7412437288 保存完了 (20/2500)
Match ID KR_7412390763 保存完了 (21/2500)
Match ID KR_7412291677 保存完了 (22/2500)
Match ID KR_7410922107 保存完了 (23/2500)
Match ID KR_7410801708 保存完了 (24/2500)
Match ID KR_7410709569 保存完了 (25/2500)
Match ID KR_7410658377 保存完了 (26/2500)
Match ID KR_741056138